In [6]:
randomize <- function(size) { 
  ## this is a dead-simple randomizer 
  ## ---   
  ## inputs: the size of the population (must be even)
  ## ---
  ## output: a vector of 1s and 0s 

  list_of_assignments <- c(rep(0, size/2), rep(1, size/2))
  randomized_assignments <- sample(list_of_assignments)
  
  return(randomized_assignments)
  } 

est_ate <- function(outcome, treat) { 
  mean(outcome[treat==1]) - mean(outcome[treat==0])
  } 

# Practice! 

We've just gone through a basic description how the p-values change when we the state of the world is that we *do* have a treatment effect (in this case it was +25) compared to when we didn't have a treatment effect (previously it was +0). 

We want you to take this point home: 

> What we want from a p-value is the following: If the null hypothesis is **not** true, we would like to have a small p-value. If the null hypothesis **is** true, we would like to have a large p-value. 

# Solidifying Understanding 

At the point, we have built *all* of the mechanics to conduct a simulation for an entire experiment. This simulation, under the *randomization inference* paradigm produces for us p-values. 

There is one piece of the discussion that we think we may have moved too quickly past -- the point that when we actually run the experiment we generate a *single* set of outcomes that we're going to use for everything related to the simulation. 

Consider the following data that we've been using through this section (3.5):

In [7]:
d <- data.frame(
    group = c(rep("Man", 20), rep("Woman", 20)), 
    po_control = c(1:20, 51:70),
    po_treat   = c(1:20, 51:70) + 25
    )

In this data, we are able to conceptualize a potential outcome to *treatment* and *control* for each individual in the experiment. After all, as we talked about last week every subject actually posesses both of these concepts, even if we get to see only one for any particular unit. 

When we actually conduct the experiment, we're **randomly** revealing one or the other of those values. If we happen to assign a unit to treatment, then we're revealing $Y_{i}(1)$ for that unit $i$. And, if we should happen to randomly put that unit into control, then we would be revealing $Y_{i}(0)$ for that unit $i$. 

One way to represent this is to think that rather than a *science table* `d`, when we actually run the experiment we end up with the *observable data*, maybe stored in a data frame `obs_d`. 

In [8]:
d$treat <- randomize(40)

obs_d <- d[ , c('group', 'treat')]
obs_d$Y <- ifelse(d$treat == 1, d$po_treat, d$po_control)

head(obs_d)

group,treat,Y
Man,0,1
Man,1,27
Man,1,28
Man,0,4
Man,1,30
Man,0,6


The good news is that our experimental design is *purpose built* to use only this observable data. 

# Simulating an Experiment from Top-to-Bottom

We have a lot of pieces built into the week right now: 

- Data that we're making 
- Potential Outcomes to Treatment 
- Potential Outcomes to Control 
- Random Assignments into Treatment and Control
- An Estimator to Calculate the ATE 
- A procedure to produce values under the sharp null. 

In [72]:
create_population <- function(pop_size, treat_effect) {
    ## create a population of outcomes for the estrogen experiment 
    ## --- 
    ## input  : pop_size    : the size of the population you want to make 
    ##        : treat_effect: the magnitude of the treatment effect 
    ## --- 
    ## output : a population science table with po to treatment and control 
    population <- data.frame(
              group = c(rep("Man", pop_size/2), rep("Woman", pop_size/2)), 
              po_control = c(sample(1:20,  size = pop_size/2, replace = TRUE), 
                             sample(51:70, size = pop_size/2, replace = TRUE))
              )
    population$po_treat <- population$po_control + treat_effect
    
    return(population)
}

randomize <- function(size) { 
    ## this is a dead-simple randomizer 
    ## ---   
    ## inputs: the size of the population (must be even)
    ## ---
    ## output: a vector of 1s and 0s 

    list_of_assignments <- c(rep(0, size/2), rep(1, size/2))
    randomized_assignments <- sample(list_of_assignments)
  
    return(randomized_assignments)
} 

run_experiment <- function(pop = population, experiment_size=experiment_size, idx=idx) { 
    ## run ONE experiment that: (a) samples from the population;
    ## then: (b) reveals potential outcomes 
    ## ---
    ## input  : pop : a population of people to draw from (made by create_population)
    ## .      : experiment_size : a named argument for the size of the experiment 
    ## .      : idx : an indicator for which people to sample 
    ## --- 
    ## output : a data frame with treatment assignment, covariates, and measured outcomes
    
    ## sample from the total population to make an experimental population
    experiment_population <- pop[idx, ] # taking the rows that are in the obj `idx` 
    ## assign some of them to treatment 
    experiment_population$treat <- randomize(size=experiment_size)
    ## keep only the covariate and treatment assignment in the `obs_d` data
    obs_d <- experiment_population[ , c('group', 'treat')]
    ## if they're in treatment, measure the po_treat, otherwise measure po_control
    obs_d$Y <- ifelse(test = experiment_population$treat == 1, 
                      yes = experiment_population$po_treat, 
                      no = experiment_population$po_control)
    
    return(obs_d)
}

est_ate <- function(outcome, treat) { 
    ## calculate a two-group difference
    mean(outcome[treat==1]) - mean(outcome[treat==0])
} 

conduct_ri <- function(nsims, dat=obs_d) { 
    ## conduct randomization inference 
    ## ---
    ## inputs  : an observed, experimental data frame 
    ##         : the number of simulations to run (1000) is more than enough
    ## returns : a distribution of ates under the sharp null 
    dist_under_sharp_null <- rep(NA, nsims)

    for(i in 1:nsims){ 
        dat$treat <- randomize(nrow(dat))
        dist_under_sharp_null[i]  <- est_ate(dat$Y, dat$treat)
    }
    
    return(dist_under_sharp_null)
}

make_conclusion <- function(estimated_ate, distribution) {  
    ## make a conclusion 
    ## ---
    ## inputs  : a distribution under the sharp null 
    ##         : the measured, two-group difference from the experiment
    ## --- 
    ## outputs : a p-value under randomization
    mean(distribution > estimated_ate)
}

In [73]:
whole_sim <- function(nsims, experiment_size, treat_effect, pop_size) { 
    population     <- create_population(pop_size=pop_size, treat_effect=treat_effect)
    experiment_idx <- sample(1:pop_size, experiment_size, replace = FALSE)
    obs_d          <- run_experiment(pop=population, experiment_size=experiment_size, idx=experiment_idx)
    estimated_ate  <- est_ate(outcome=obs_d$Y, treat=obs_d$treat)
    dist_under_sharp_null <- conduct_ri(nsims)
    p_val          <- make_conclusion(estimated_ate=estimated_ate, distribution=dist_under_sharp_null)
    
    return(list(ate = estimated_ate, p_val = p_val))
    
}

Let's suppose that there is a really large population of people that we could *possibly* bring into experiment. Rather than just 40 people who are under consideration, instead there are 4,000. 

We can make data that is similar to what we've had before, just expanded somewhat. 

In [77]:
whole_sim(nsims=1000, experiment_size=40, treat_effect=20, pop_size=4000)

$ate
[1] 13.4

$p_val
[1] 0.061

In [80]:
getwd()

[1] "/Users/alehughe/Desktop/2u_revisions"